In [1]:
import os
%pwd
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class model_eval_config:
    root_dir: Path




In [2]:
from src.Mini_Translator_T.constants import *
from src.Mini_Translator_T.utils.common import read_yaml, create_directories


In [3]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_eval_model_config(self) -> model_eval_config:
        config = self.config.model_evaluation
        params=self.params

        create_directories([config.root_dir])

        eval_config=model_eval_config(root_dir=config.root_dir)
        
        return eval_config


NameError: name 'model_eval_config' is not defined

In [5]:
import torch
import numpy as np
import torch.nn as nn
from src.Mini_Translator_T.logging import logger
import pandas as pd
from src.Mini_Translator_T.config.configuration import model_trainer_config
from src.Mini_Translator_T.utils.common import casual_mask
from tokenizers import Tokenizer

c:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-24 11:10:42,542: INFO: config: PyTorch version 2.0.0 available.]
[2024-05-24 11:10:42,554: INFO: config: TensorFlow version 2.16.1 available.]


In [10]:
class Evaluation:
    def __init__(self,config:model_eval_config,config2=model_trainer_config):

        
        self.config=config
        self.config2=config2



    # Define function to obtain the most probable next token
    def greedy_decode(self,model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
        # Retrieving the indices from the start and end of sequences of the target tokens
        sos_idx = tokenizer_tgt.token_to_id('[SOS]')
        eos_idx = tokenizer_tgt.token_to_id('[EOS]')

        # Computing the output of the encoder for the source sequence
        encoder_output = model.encode(source, source_mask)
        # Initializing the decoder input with the Start of Sentence token
        decoder_input = torch.empty(1,1).fill_(sos_idx).type_as(source).to(device)

        # Looping until the 'max_len', maximum length, is reached
        while True:
            if decoder_input.size(1) == max_len:
                break

            # Building a mask for the decoder input
            decoder_mask = casual_mask(decoder_input.size(1)).type_as(source_mask).to(device)

            # Calculating the output of the decoder
            out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

            # Applying the projection layer to get the probabilities for the next token
            prob = model.project(out[:, -1])

            # Selecting token with the highest probability
            _, next_word = torch.max(prob, dim=1)
            decoder_input = torch.cat([decoder_input, torch.empty(1,1). type_as(source).fill_(next_word.item()).to(device)], dim=1)

            # If the next token is an End of Sentence token, we finish the loop
            if next_word == eos_idx:
                break

        return decoder_input.squeeze(0) # Sequence of tokens generated by the decoder




    # Defining function to evaluate the model on the validation dataset
    # num_examples = 2, two examples per run
    def run_validation(self,model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device num_examples=2):
        model.eval() # Setting model to evaluation mode
        count = 0 # Initializing counter to keep track of how many examples have been processed

        console_width = 80 # Fixed witdh for printed messages

        # Creating evaluation loop
        with torch.no_grad(): # Ensuring that no gradients are computed during this process
            for batch in validation_ds:
                count += 1
                encoder_input = batch['encoder_input'].to(device)
                encoder_mask = batch['encoder_mask'].to(device)

                # Ensuring that the batch_size of the validation set is 1
                assert encoder_input.size(0) ==  1, 'Batch size must be 1 for validation.'

                # Applying the 'greedy_decode' function to get the model's output for the source text of the input batch
                model_out = self.greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

                # Retrieving source and target texts from the batch
                source_text = batch['src_text'][0]
                target_text = batch['tgt_text'][0] # True translation
                model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy()) # Decoded, human-readable model output

                # Printing results
                logger.info('-'*console_width)
                logger.info(f'SOURCE: {source_text}')
                logger.info(f'TARGET: {target_text}')
                logger.info(f'PREDICTED: {model_out_text}')

                # After two examples, we break the loop
                if count == num_examples:
                    break


    def initiate_model_trainer(self):
        config=self.config2
        root_dir = "artifacts/data_transformation"
        valid_data_loader_path = os.path.join(root_dir, "valid_data_loader.pth")

        
        tokenizer_en = os.path.join(root_dir, "'tokenizer_en.json'")
        tokenizer_it = os.path.join(root_dir, "'tokenizer_it.json'")
        tokenizer_src = Tokenizer.from_file(tokenizer_en)
        tokenizer_tgt = Tokenizer.from_file(tokenizer_it)



        # Load the DataLoader objects
        valid_data_loader = torch.load(valid_data_loader_path)
      


        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Using device {device}")

        # Creating model directory to store weights
        Path(config.model_folder).mkdir(parents=True, exist_ok=True)

        # Retrieving dataloaders and tokenizers for source and target languages using the 'get_ds' function

        # Initializing model on the GPU using the 'get_model' function
        # Load the entire model
        model = torch.load('full_model.pth')


        self.run_validation(model, valid_data_loader, tokenizer_src, tokenizer_tgt, config.seq_len,device)

  


In [11]:
try:
    config = ConfigurationManager()
    eval_model_config = config.get_eval_model_config()
    model = Evaluation(config=eval_model_config)
    model.initiate_model_Evaluation()
except Exception as e:
    raise e

[2024-05-24 11:19:16,181: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-24 11:19:16,191: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-24 11:19:16,196: INFO: common: created directory at: artifacts]
[2024-05-24 11:19:16,201: INFO: common: created directory at: artifacts/evaluation]
[2024-05-24 11:19:16,211: INFO: common: yaml file: params.yaml loaded successfully]
4
5
6
7
8
9
10
11
| Test Loss: 5.823 | Test PPL: 337.897 |


AttributeError: 'WindowsPath' object has no attribute 'data_transformation'